# **1\. CREATING INITIAL TABLES**

**All information in this project is acquired from the Faculty of Computer Science and Engineering's official website [https://www.finki.ukim.mk/](https:\www.finki.ukim.mk%29%29\) and the courses platform [https://courses.finki.ukim.mk/](https:\courses.finki.ukim.mk%29%29\).**

In [53]:
DROP TABLE IF EXISTS MandatoryCourseStudies, CourseProfessors, Studies, Courses, Professors

Commands completed successfully.

Total execution time: 00:00:00.061

### **1.1. Professors table**

The Professors table contains data about all of the professors whose information is currently accessible online. The table contains information about every professor's/assistant's fist and last name, email and the total number of courses the professor/assistant is involved in.

In [54]:
DROP TABLE IF EXISTS Professors;
CREATE TABLE Professors(
    profCode INT PRIMARY KEY,
    firstName NVARCHAR(50) COLLATE Cyrillic_General_CI_AS, 
    lastName NVARCHAR(50) COLLATE Cyrillic_General_CI_AS,
    email NVARCHAR(50),
    numberOfCourses INT DEFAULT 0
);

Commands completed successfully.

Total execution time: 00:00:00.048

### **1.2. Studies table**

The Studies table contains data about the name of the study and how many years it takes to complete it. Every study also has a distinct code/key (studiesCode).

In [55]:
DROP TABLE IF EXISTS Studies;
CREATE TABLE Studies(
    studiesCode INT PRIMARY KEY,
    name NVARCHAR(70) COLLATE Cyrillic_General_CI_AS,
    yearsToComplete NVARCHAR(15),
);

Commands completed successfully.

Total execution time: 00:00:00.041

### **1.3. Courses table**

The courses table contains a code/key for each course and it's name.

In [56]:
DROP TABLE IF EXISTS Courses;
CREATE TABLE Courses(
    coursesCode NVARCHAR(10) PRIMARY KEY,
    name NVARCHAR(70) COLLATE Cyrillic_General_CI_AS,
);

Commands completed successfully.

Total execution time: 00:00:00.041

### **1.4. CourseProfessors table**

This table contains combinations of professor and course codes. For example if one entry contains professor code A and course code B means that professor A is involved in the course B (according to the information listed online). Both of the codes are references to the corresponding codes in the professors and courses tables. If a code is deleted or updated in the courses or professors table, it will be deleted or updated in this table too.

In [57]:
DROP TABLE IF EXISTS CourseProfessors;
CREATE TABLE CourseProfessors(
    coursesCode NVARCHAR(10),
    profCode INT,
    CONSTRAINT CourseProfessors_PK PRIMARY KEY(coursesCode, profCode),
    CONSTRAINT CP_CourseCode_FK FOREIGN KEY (coursesCode) REFERENCES Courses(coursesCode) 
        ON DELETE CASCADE 
        ON UPDATE CASCADE,
    CONSTRAINT CP_StudiesCode_FK FOREIGN KEY (profCode) REFERENCES Professors(profCode) 
        ON DELETE CASCADE
        ON UPDATE CASCADE
)

Commands completed successfully.

Total execution time: 00:00:00.042

### **1.5. MandatoryCourseStudies table**

This table contains the studies codes and the corresponding mandatory courses codes for each study along with the semester they are predicted to be studied in. The study and course codes are foreign keys and point to data in the Courses and Studies tables. Whenever an entry is updated or deleted in those tables, the same happens in this table.

In [58]:
DROP TABLE IF EXISTS MandatoryCourseStudies;
CREATE TABLE MandatoryCourseStudies(
    studiesCode INT,
    coursesCode NVARCHAR(10),
    semester INT,
    CONSTRAINT CourseStudies_PK PRIMARY KEY(coursesCode, studiesCode),
    CONSTRAINT MCS_CourseCode_FK FOREIGN KEY (coursesCode) REFERENCES Courses(coursesCode) 
    ON DELETE CASCADE
    ON UPDATE CASCADE,
    CONSTRAINT MCS_StudiesCode_FK FOREIGN KEY (studiesCode) REFERENCES Studies(studiesCode) 
    ON DELETE CASCADE
    ON UPDATE CASCADE,
)

Commands completed successfully.

Total execution time: 00:00:00.042

In [59]:
DROP TABLE IF EXISTS ProfessorsStatistics, CourseStatistics, StudiesStatistics

Commands completed successfully.

Total execution time: 00:00:00.008

### **1.6. ProfessorsStatistics table**

This table contains the total number of professors and assistants employed at the faculty and the average number of courses each professor is involved in.

In [60]:
DROP TABLE IF EXISTS ProfessorsStatistics
CREATE TABLE ProfessorsStatistics (
    totalNumberOfProfessors INT,
    avgNumOfMandatoryCoursesPerProfessor FLOAT,
);

Commands completed successfully.

Total execution time: 00:00:01.214

### **1.7. CourseStatistics table**

The CourseStatistics table contains the total number of courses available at the faculty, how many of them are mandatory for at least one study and how many are elective courses.

In [61]:
DROP TABLE IF EXISTS CourseStatistics
CREATE TABLE CourseStatistics (
    totalNumberOfCourses INT,
    mandatoryCoursesCount INT,
    electiveCoursesCount INT,
)

Commands completed successfully.

Total execution time: 00:00:00.040

### **1.8. StudiesStatistics table**

This table contains information about the total number of studies available at the faculty, what is the average number of mandatory courses per semester, how many two, three and four year studies there are, accordingly.

In [62]:
DROP TABLE IF EXISTS StudiesStatistics
CREATE TABLE StudiesStatistics (
    totalNumberOfStudies INT,
    avgNumMandatoryCoursesPerSemester FLOAT,
    totalNumberOfTwoYearStudies INT,
    totalNumberOfThreeYearStudies INT,
    totalNumberOfFourYearStudies INT
)

Commands completed successfully.

Total execution time: 00:00:00.041

# **2\. SETTING UP TRIGGERS**

In [63]:
DROP TRIGGER IF EXISTS UpdateProfessorsNumberOfCourses

Commands completed successfully.

Total execution time: 00:00:00

### **2.1. UpdateProfessorsNumberOfCourses trigger**

This trigger is responsible for updating the numberOfCourses value in the Professors table. After each insert, delete or update of an entry in the CourseProfessors table, this trigger updates the number of courses the certain professor is involved in.

In [64]:
CREATE TRIGGER UpdateProfessorsNumberOfCourses
On CourseProfessors
AFTER INSERT, DELETE, UPDATE
AS
BEGIN
    UPDATE Professors
    SET numberOfCourses = (
        SELECT COUNT(DISTINCT profCode) 
        FROM CourseProfessors
        WHERE CourseProfessors.profCode = Professors.profCode
    )
END;

Commands completed successfully.

Total execution time: 00:00:00.026

In [65]:
DROP TRIGGER IF EXISTS UpdateProfessorsStatistics

Commands completed successfully.

Total execution time: 00:00:00.001

### **2.2. UpdateProfessorsStatistics trigger**

This trigger is designed to automatically update the statistics related to professors stored in the ProfessorsStatistics table whenever there is an insertion, deletion, or update operation perdormed on the Professors table. After modifications this trigger recalculates the average number of mandatory courses per professorand and total number of professors, and updates the corresponding values in the table.

In [66]:
CREATE TRIGGER UpdateProfessorsStatistics
ON Professors
AFTER INSERT, DELETE, UPDATE
AS
BEGIN
    UPDATE ProfessorsStatistics 
    SET avgNumOfMandatoryCoursesPerProfessor = (
        SELECT COALESCE(AVG(numberOfCourses), 0) FROM Professors
    );

    UPDATE ProfessorsStatistics 
    SET totalNumberOfProfessors = (
        SELECT COUNT(DISTINCT profCode) FROM Professors
    );
END;

Commands completed successfully.

Total execution time: 00:00:00.040

In [67]:
DROP TRIGGER IF EXISTS UpdateStudiesStatistics

Commands completed successfully.

Total execution time: 00:00:00

### **2.3. UpdateStudiesStatistics trigger**

This trigger keeps track of key statistics related to studies in the database. It activates after any change in the studies table and updates the StudiesStatistics table accordingly. It calculates and updates fields for the total number of studies, the average number of mandatory courses per semester, and the distribution of studies by completion duration (two, three, or four years).

In [68]:
CREATE TRIGGER UpdateStudiesStatistics
ON Studies
AFTER INSERT, DELETE, UPDATE
AS
BEGIN
    UPDATE StudiesStatistics
    SET totalNumberOfStudies = (
        SELECT COUNT(DISTINCT studiesCode) 
        FROM Studies
    );

    UPDATE StudiesStatistics
    SET avgNumMandatoryCoursesPerSemester = (
        SELECT AVG(numMandatoryCourses) 
        FROM (
            SELECT COUNT(coursesCode) AS numMandatoryCourses
            FROM MandatoryCourseStudies
            GROUP BY semester
        ) AS avgs
    );

    UPDATE StudiesStatistics
    SET totalNumberOfTwoYearStudies = (
        SELECT COUNT(studiesCode) 
        FROM Studies 
        WHERE yearsToComplete = 2
    );

    UPDATE StudiesStatistics
    SET totalNumberOfThreeYearStudies = (
        SELECT COUNT(studiesCode) 
        FROM Studies 
        WHERE yearsToComplete = 3
    );

    UPDATE StudiesStatistics
    SET totalNumberOfFourYearStudies = (
        SELECT COUNT(studiesCode) 
        FROM Studies 
        WHERE yearsToComplete = 4
    );
END;


Commands completed successfully.

Total execution time: 00:00:00.043

In [69]:
DROP PROCEDURE IF EXISTS UpdateCourseStatistics;

Commands completed successfully.

Total execution time: 00:00:00.041

### **2.4. UpdateCourseStatistics procedure**

This procedure calculates and updates statistics related to the courses in the database. It updates the CourseStatistics table with the total number of courses, the count of mandatory courses, and the count of elective courses. This procedure is designed in order to avoid code repetition in the following two triggers.

In [70]:
CREATE PROCEDURE UpdateCourseStatistics
AS
BEGIN
    DECLARE @totalNumberOfCourses INT;
    DECLARE @mandatoryCoursesCount INT;

    SELECT @totalNumberOfCourses = COUNT(coursesCode) FROM Courses;

    SELECT @mandatoryCoursesCount = COUNT(DISTINCT coursesCode) FROM MandatoryCourseStudies;

    UPDATE CourseStatistics
    SET totalNumberOfCourses = @totalNumberOfCourses,
        mandatoryCoursesCount = @mandatoryCoursesCount,
        electiveCoursesCount = @totalNumberOfCourses - @mandatoryCoursesCount;
END;

Commands completed successfully.

Total execution time: 00:00:00.041

In [71]:
DROP TRIGGER IF EXISTS UpdateCourseStatisticsOnCoursesAlter, UpdateCourseStatisticsOnMandatoryCourseStudiesAlter

Commands completed successfully.

Total execution time: 00:00:00

### **2.5. UpdateCourseStatisticsOnCoursesAlter**

This trigger executes the UpdateCourseStatistics procedure after aterations in the Courses table.

In [72]:
CREATE TRIGGER UpdateCourseStatisticsOnCoursesAlter
ON Courses
AFTER INSERT, DELETE, UPDATE
AS
BEGIN
    EXEC UpdateCourseStatistics;
END;

Commands completed successfully.

Total execution time: 00:00:00.001

### **2.6. UpdateCourseStatisticsOnMandatoryCourseStudiesAlter trigger**

This trigger executes the UpdateCourseStatistics procedure after aterations in the MandatoryCourseStudies table

In [73]:
CREATE TRIGGER UpdateCourseStatisticsOnMandatoryCourseStudiesAlter
ON MandatoryCourseStudies
AFTER INSERT, DELETE, UPDATE
AS
BEGIN
    EXEC UpdateCourseStatistics;
END;

Commands completed successfully.

Total execution time: 00:00:00.040

# **2\. POPULATING THE TABLES**

In [74]:
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (1, N'Марјан', N'Гушев', 'marjan.gushev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (2, N'Катерина', N'Здравкова', 'katerina.zdravkova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (3, N'Сузана', N'Лошковска', 'suzana.loshkovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (4, N'Коста', N'Митрески', 'kosta.mitreski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (5, N'Верица', N'Бакева', 'verica.bakeva@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (6, N'Владимир', N'Трајковиќ', 'vladimir.trajkovik@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (7, N'Ана', N'Мадевска Богданова', 'ana.madevska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (8, N'Дејан', N'Ѓорѓевиќ', 'dejan.gjorgjevikj@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (9, N'Димитар', N'Трајанов', 'dimitar.trajanov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (10, N'Андреа', N'Кулаков', 'andrea.kulakov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (11, N'Љупчо', N'Антовски', 'ljupcho.antovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (12, N'Марија', N'Михова', 'marija.mihova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (13, N'Слободан', N'Калајџиски', 'slobodan.kalajdziski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (14, N'Невена', N'Ацковска', 'nevena.ackovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (15, N'Горан', N'Велинов', 'goran.velinov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (16, N'Анастас', N'Мишев', 'anastas.mishev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (17, N'Соња', N'Филипоска', 'sonja.filiposka@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (18, N'Иван', N'Чорбев', 'ivan.chorbev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (19, N'Ласко', N'Баснарков', 'lasko.basnarkov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (20, N'Боро', N'Јакимовски', 'boro.jakimovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (21, N'Весна', N'Димитрова', 'vesna.dimitrova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (22, N'Гоце', N'Арменски', 'goce.armenski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (23, N'Соња', N'Гиевска', 'sonja.gievska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (24, N'Дејан', N'Спасов', 'dejan.spasov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (25, N'Ивица', N'Димитровски', 'ivica.dimitrovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (26, N'Игор', N'Мишковски', 'igor.mishkovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (27, N'Ѓорѓи', N'Маџаров', 'gjorgji.madjarov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (28, N'Смилка', N'Јанеска-Саркањац', 'smilka.janeska.sarkanjac@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (29, N'Вангел', N'Ајановски', 'vangel.ajanovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (30, N'Весна', N'Димитриевска Ристовска', 'vesna.dimitrievska.ristovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (31, N'Миле', N'Јованов', 'mile.jovanov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (32, N'Билјана', N'Стојкоска', 'biljana.stojkoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (33, N'Кире', N'Триводалиев', 'kire.trivodaliev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (34, N'Сашо', N'Граматиков', 'sasho.gramatikov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (35, N'Мирослав', N'Мирчев', 'miroslav.mirchev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (36, N'Георгина', N'Мирчева', 'georgina.mircheva@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (37, N'Магдалена', N'Костоска', 'magdalena.kostoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (38, N'Александра', N'Поповска Митровиќ', 'aleksandra.popovska.mitrovikj@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (39, N'Наташа', N'Илиевска', 'natasha.ilievska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (40, N'Билјана', N'Тојтовска', 'biljana.tojtovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (41, N'Милош', N'Јовановиќ', 'milos.jovanovik@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (42, N'Андреја', N'Наумоски', 'andreja.naumoski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (43, N'Панче', N'Рибарски', 'pance.ribarski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (44, N'Христина', N'Михајлоска', 'hristina.mihajloska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (45, N'Иван', N'Китановски', 'ivan.kitanovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (46, N'Ефтим', N'Здравевски', 'eftim.zdravevski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (47, N'Петре', N'Ламески', 'petre.lameski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (48, N'Катарина', N'Тројачанец Динева', 'katarina.trojacanec@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (49, N'Владимир', N'Здравески', 'vladimir.zdraveski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (50, N'Бојана', N'Котеска', 'bojana.koteska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (51, N'Моника', N'Симјаноска', 'monika.simjanoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (52, N'Методија', N'Јанчески', 'metodija.jancheski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (53, N'Ристе', N'Стојанов', 'riste.stojanov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (54, N'Александра', N'Дединец', 'aleksandra.dedinec@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (55, N'Емил', N'Станков', 'emil.stankov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (56, N'Борис', N'Станоев', 'boris.stanoev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (57, N'Бобан', N'Јоксимоски', 'boban.joksimoski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (58, N'Илинка', N'Иваноска', 'ilinka.ivanoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (59, N'Бојан', N'Илијоски', 'bojan.ilijoski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (60, N'Костадин', N'Мишев', 'kostadin.mishev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (61, N'Александар', N'Тенев', 'aleksandar.tenev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (62, N'Александар', N'Стојменски', 'aleksandar.stojmenski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (63, N'Ненад', N'Анчев', 'nenad.anchev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (64, N'Војдан', N'Ќорвезироски', 'vojdan.kjorveziroski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (65, N'Мартина', N'Тошевска', 'martina.toshevska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (66, N'Влатко', N'Спасев', 'vlatko.spasev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (67, N'Петар', N'Секулоски', 'petar.sekuloski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (68, N'Стефан', N'Андонов', 'stefan.andonov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (69, N'Кирил', N'Ќироски', 'kiril.kjiroski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (70, N'Ана', N'Тодоровска', 'ana.todorovska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (71, N'Јована', N'Добрева', 'jovana.dobreva@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (72, N'Јана', N'Кузманова', 'jana.kuzmanova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (73, N'Димитар', N'Китановски', 'dimitar.kitanovski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (74, N'Томе', N'Ефтимов', 'tome.eftimov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (75, N'Андреј', N'Ристески', 'andrej.risteski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (76, N'Славе', N'Темков', 'slave.temkov@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (77, N'Милена', N'Трајаноска', 'milena.trajanoska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (78, N'Живко', N'Атанаскоски', 'zhivko.atanaskoski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (79, N'Димитар', N'Милески', 'dimitar.mileski@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (80, N'Зорица', N'Карапанчева', 'zorica.karapancheva@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (81, N'Марија', N'Танеска', 'marija.taneska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (82, N'Мила', N'Додевска', 'mila.dodevska@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (83, N'Ана', N'Атанасова', 'ana.atanasova@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (84, N'Александар', N'Петрушев', 'aleksandar.petrushev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (85, N'Мартин', N'Динев', 'martin.dinev@finki.ukim.mk');
INSERT INTO Professors(profCode, firstName, lastName, email) VALUES (86, N'Владислав', N'Бидиков', 'vladislav.bidikov@finki.ukim.mk');

Commands completed successfully.

Total execution time: 00:00:00.058

In [75]:
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (0, N'Интернет, мрежи и безбедност', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (1, N'Интернет, мрежи и безбедност', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (2, N'Компјутерска едукација', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (3, N'Компјутерски науки', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (4, N'Компјутерски науки', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (5, N'Компјутерско инженерство', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (6, N'Компјутерско инженерство', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (7, N'Примена на информациски технологии', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (8, N'Примена на информациски технологии', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (9, N'Софтверско инженерство и информациски системи', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (10, N'Софтверско инженерство и информациски системи', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (11, 'Software engineering and information systems', '3');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (12, 'Software engineering and information systems', '4');

Commands completed successfully.

Total execution time: 00:00:00.031

In [76]:
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W005', N'Бизнис и менаџмент');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W007', N'Вовед во компјутерските науки');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S013', N'Калкулус');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W018', N'Професионални вештини');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W020', N'Структурно програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S003', N'Архитектура и организација на компјутери');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W011', N'Дискретна математика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S015', N'Објектно ориентирана анализа и дизајн');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S016', N'Објектно-ориентирано програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('0', N'Спорт и здравје');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S034', N'Калкулус 2');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S116', N'Компјутерски компоненти');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S120', N'Креативни вештини за решавање проблеми');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S026', N'Маркетинг');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S146', N'Основи на веб дизајн');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S066', N'Основи на сајбер безбедноста');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W001', N'Алгоритми и податочни структури');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W006', N'Веројатност и статистика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W014', N'Компјутерски мрежи и безбедност');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S100', N'Економија за ИКТ инженери');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W104', N'Инженерска математика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W109', N'Интернет програмирање на клиентска страна');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W140', N'Напредно програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W165', N'Управување со техничката поддршка');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W167', N'Шаблони за дизајн на кориснички интерфејси');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S002', N'Анализа на софтверските барања');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S017', N'Оперативни системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S061', N'Безжични и мобилни системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S030', N'Вештачка интелигенција');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S082', N'Визуелно програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S084', N'Вовед во екоинформатиката');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S090', N'Вовед во случајни процеси');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S095', N'Дигитално процесирање на слика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S097', N'Дизајн на алгоритми');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S099', N'Е-влада');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S042', N'Електрични кола');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S110', N'Интернет технологии');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S114', N'Компјутерска графика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S119', N'Концепти на информатичко општество');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S124', N'Медиуми и комуникации');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S164', N'Теорија на информации со дигитални комуникации');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W004', N'Бази на податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W008', N'Вовед во науката за податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W009', N'Дизајн и архитектура на софтвер');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W060', N'Администрација на системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W024', N'Веб програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W081', N'Визуелизација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W043', N'Информациска безбедност');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W053', N'Компјутерска етика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W115', N'Компјутерски звук, говор и музика ');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W035', N'Линеарна алгебра и примени');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W134', N'Мултимедиски мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W136', N'Напреден веб дизајн');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W148', N'Основи на роботиката');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S010', N'Дизајн на интеракцијата човек-компјутер');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S012', N'Интегрирани системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S019', N'Софтверски квалитет и тестирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S073', N'Агентно-базирани системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S077', N'Безжични мултимедиски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S040', N'Вградливи микропроцесорски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S062', N'Виртуелизација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S087', N'Вовед во мрежна наука');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S091', N'Географски информациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S093', N'Дигитална форензика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S025', N'Електронска и мобилна трговија');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S118', N'Континуирана интеграција и испорака');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S036', N'Машинско учење');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S135', N'Мултимедиски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S149', N'Паралелно програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S150', N'Податочно рударство');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S047', N'Процесирање на сигналите');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S155', N'Сервисно ориентирани архитектури');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S159', N'Софтверски дефинирана безбедност');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S163', N'Статистичко моделирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S039', N'Формални јазици и автомати');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W021', N'Тимски проект');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W072', N'Автономна роботика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W074', N'Администрација на бази на податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W075', N'Анализа и дизајн на ИС');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W079', N'Веб базирани системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W085', N'Вовед во биоинформатиката');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W088', N'Вовед во паметни градови');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W089', N'Вовед во препознавање на облици');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W092', N'Дигитална постпродукција');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W103', N'Имплементација на системи со слободен и отворен код');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W105', N'Иновации во ИКТ');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W108', N'Интернет на нештата');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W123', N'Машинска визија');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W027', N'Менаџмент на информациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W126', N'Методологија на истражувањето во ИКТ');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W128', N'Мобилни информациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W129', N'Мобилни платформи и програмирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W137', N'Напредна интеракција човек компјутер');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W142', N'Обработка на природните јазици');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W144', N'Операциони истражувања');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W068', N'Пресметување во облак');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W152', N'Програмирање на видео игри');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W038', N'Програмски парадигми');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W154', N'Рударење на масивни податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W156', N'Системи за поддршка при одлучувањето');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W048', N'Софтвер за вградливи системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W160', N'Софтверски дефинирани мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W161', N'Социјални мрежи и медиуми');
INSERT INTO Courses(coursesCode, name) VALUES ('1', N'Дипломска работа');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S022', N'Управување со ИКТ проекти');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S07', N'Биолошки инспирирано пресметување');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S121', N'Блоковски вериги и криптовалути');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S080', N'Веб пребарувачки системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S083', N'Виртуелна реалност');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S086', N'Вовед во когнитивни науки');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S102', N'ИКТ за развој');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S106', N'Интелигентни информациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S107', N'Интелигентни системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S113', N'Компјутерска анимација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S127', N'Мобилни апликации');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S130', N'Моделирање и менаџирање на бизнис процеси');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S132', N'Модерни трендови во роботика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S138', N'Напредни бази на податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S141', N'Неструктурирани бази на податоци');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S028', N'Претприемништво');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S157', N'Складови на податоци и аналитичка обработка');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S162', N'Споделување и пресметување во толпа');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W070', N'Педагогија');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S052', N'ИТ системи за учење');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S045', N'Компјутерски архитектури');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S023', N'Бизнис статистика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W055', N'Мултимедијални технологии');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W096', N'Дигитизација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W050', N'Дизајн на образовен софтвер');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S056', N'Персонализирано учење');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S071', N'Психологија на училишна возраст');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S057', N'Работа со надарени ученици');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S094', N'Дигитални библиотеки');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S166', N'Учење на далечина');
INSERT INTO Courses(coursesCode, name) VALUES ('2', N'Самостоен проект');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S069', N'Македонски јазик');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S054', N'Методика на информатиката');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W031', N'Дискретни структури 1');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1S032', N'Дискретни структури 2');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W037', N'Паралелно и дистрибуирано процесирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S125', N'Мерење и анализа на интернет сообраќај');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S153', N'Процесна роботика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S158', N'Современи компјутерски архитектури');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W145', N'Оптички мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S151', N'Пресметковна биологија');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S112', N'Програмски јазици и компајлери');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W049', N'Физика');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W147', N'Основи на комуникациски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S143', N'Одржливи и енергетски ефикасни компјутерски системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W044', N'Компјутерска електроника');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W131', N'Моделирање и симулација');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S101', N'Етичко хакирање');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W046', N'Компјутерски мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2W067', N'Основи на теоријата на информации');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S029', N'Софтверско инженерство');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W033', N'Калкулус 1');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L1W041', N'Дизајн на дигитални кола');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L2S051', N'ИКТ во образованието');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S063', N'Дизајн на компјутерски мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W064', N'Дистрибуирани системи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3S059', N'Администрација на мрежи');
INSERT INTO Courses(coursesCode, name) VALUES ('F18L3W065', N'Мрежна безбедност');


Commands completed successfully.

Total execution time: 00:00:00.157

In [77]:
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L1S066', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2W046', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2W067', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2S061', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L3W060', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, 'F18L3W065', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (0, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L1S066', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2W046', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2W067', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2S061', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W060', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W065', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3S059', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3S062', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W064', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3W068', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, '1', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (1, 'F18L3S063', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W011', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W070', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1S052', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1S013', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3W055', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L2S051', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3W050', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3W053', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S056', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S071', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S057', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, '2', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, '1', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S069', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (2, 'F18L3S054', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W031', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W033', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1S032', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1S034', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2S030', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, 'F18L3S036', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (3, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W031', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W033', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1S032', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1S034', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2S030', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3W035', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3W037', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3S036', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3S039', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, 'F18L3W038', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (4, '1', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W041', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W033', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W049', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1S034', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2W046', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2S042', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, 'F18L3S040', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (5, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W041', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W033', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W049', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1S034', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1S045', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2W046', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2S042', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3W043', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3W044', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3S040', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3S047', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3W048', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, 'F18L3S022', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (6, '1', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W011', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1S023', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1S026', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L3W024', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (7, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W011', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1S023', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1S026', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L2S029', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W024', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W008', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3S025', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W027', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3W021', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, '1', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3S028', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (8, 'F18L3S022', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1S015', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2S002', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L3W009', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, 'F18L3S019', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (9, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1S015', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2S002', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3W008', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3W009', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3S012', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3S019', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3W021', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, 'F18L3S022', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (10, '1', 8);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1S015', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2S002', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L3W009', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, 'F18L3S019', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (11, '1', 6);

INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W005', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W007', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1S013', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W018', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W020', 1);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1S003', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1W011', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1S015', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L1S016', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, '0', 2);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2W001', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2W006', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2W014', 3);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2S002', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L2S017', 4);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3W004', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3W008', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3W009', 5);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3S010', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3S012', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3S019', 6);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3W021', 7);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, 'F18L3S022', 8);
INSERT INTO MandatoryCourseStudies(studiesCode, coursesCode, semester) VALUES (12, '1', 8);

Commands completed successfully.

Total execution time: 00:00:01.056

In [78]:
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W005', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W005', 28);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 3);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 2);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 4);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 45);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W007', 43);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 21);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S013', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 19);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 42);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 37);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W018', 43);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 18);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 8);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 25);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 44);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 27);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W020', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W011', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S045', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 18);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 8);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 25);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 44);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S016', 27);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S066', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S066', 23);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 13);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 37);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 44);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W001', 6);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W046', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W046', 1);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W046', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W067', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W067', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W067', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S061', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S061', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 9);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 21);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 26);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 44);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 41);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S017', 20);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W060', 43);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W060', 20);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 13);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 23);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 15);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 46);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W004', 29);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W065', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W065', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S059', 54);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S062', 20);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W064', 6);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W064', 47);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W064', 20);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W068', 1);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S063', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S063', 26);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S063', 54);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S063', 35);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S054', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S052', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W055', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S051', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S051', 22);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W050', 22);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W053', 2);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S056', 7);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W031', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W031', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W033', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 26);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 37);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 35);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 54);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S003', 1);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S032', 24);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S034', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S034', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S034', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S034', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2W006', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 10);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 23);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 2);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 47);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S030', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S029', 8);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S029', 2);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S029', 25);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S029', 27);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 30);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W035', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W037', 1);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W037', 46);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W037', 37);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W037', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S010', 3);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S010', 23);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S010', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S010', 45);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S036', 35);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S036', 54);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S039', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W038', 45);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W038', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W038', 31);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W041', 21);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W041', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1W049', 19);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S116', 42);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S116', 4);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S120', 6);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S120', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S026', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S026', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S026', 42);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S026', 28);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S146', 18);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S146', 13);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S146', 22);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S146', 45);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S066', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S042', 4);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W043', 21);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W044', 19);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S040', 14);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S040', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S047', 47);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S047', 19);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W048', 26);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W048', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W048', 35);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 12);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 38);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 40);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 39);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S023', 5);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 9);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 41);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 43);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S022', 49);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S028', 6);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 28);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 41);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 15);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 4);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W027', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S025', 9);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S025', 22);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S025', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S025', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W024', 9);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W024', 34);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W024', 41);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S015', 13);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S015', 32);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L1S015', 33);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S002', 17);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S002', 36);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L2S002', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W009', 27);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W009', 11);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3W009', 8);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S012', 18);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S019', 16);
INSERT INTO CourseProfessors(coursesCode, profCode) VALUES ('F18L3S019', 44);

Commands completed successfully.

Total execution time: 00:00:01.868

# **3\. TABLE RESULTS VISUELIZATION**

In [79]:
SELECT * FROM Professors

Commands completed successfully.

Total execution time: 00:00:00.109

profCode,firstName,lastName,email,numberOfCourses
1,Марјан,Гушев,marjan.gushev@finki.ukim.mk,1
2,Катерина,Здравкова,katerina.zdravkova@finki.ukim.mk,1
3,Сузана,Лошковска,suzana.loshkovska@finki.ukim.mk,1
4,Коста,Митрески,kosta.mitreski@finki.ukim.mk,1
5,Верица,Бакева,verica.bakeva@finki.ukim.mk,1
6,Владимир,Трајковиќ,vladimir.trajkovik@finki.ukim.mk,1
7,Ана,Мадевска Богданова,ana.madevska@finki.ukim.mk,1
8,Дејан,Ѓорѓевиќ,dejan.gjorgjevikj@finki.ukim.mk,1
9,Димитар,Трајанов,dimitar.trajanov@finki.ukim.mk,1
10,Андреа,Кулаков,andrea.kulakov@finki.ukim.mk,1


In [80]:
SELECT * FROM Studies

Commands completed successfully.

Total execution time: 00:00:00.051

studiesCode,name,yearsToComplete
0,"Интернет, мрежи и безбедност",3
1,"Интернет, мрежи и безбедност",4
2,Компјутерска едукација,4
3,Компјутерски науки,3
4,Компјутерски науки,4
5,Компјутерско инженерство,3
6,Компјутерско инженерство,4
7,Примена на информациски технологии,3
8,Примена на информациски технологии,4
9,Софтверско инженерство и информациски системи,3


In [81]:
SELECT * FROM MandatoryCourseStudies

Commands completed successfully.

Total execution time: 00:00:00.132

studiesCode,coursesCode,semester
0,0,2
1,0,2
2,0,2
3,0,2
4,0,2
5,0,2
6,0,2
7,0,2
8,0,2
9,0,2


In [82]:
SELECT * FROM CourseProfessors

Commands completed successfully.

Total execution time: 00:00:00.055

coursesCode,profCode
F18L1S003,1
F18L1S003,11
F18L1S003,24
F18L1S003,26
F18L1S003,32
F18L1S003,34
F18L1S003,35
F18L1S003,37
F18L1S003,49
F18L1S003,54


# **4\. TABLE MANIPULATION**

In [83]:
SET NOCOUNT ON;

DELETE FROM Studies
WHERE studiesCode = 2;

Commands completed successfully.

Total execution time: 00:00:00.082

In [84]:
SET NOCOUNT ON;

INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (13, 'Информатичка едукација', '4');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (14, 'Стручни студии за програмирање', '2');
INSERT INTO Studies(studiesCode, name, yearsToComplete) VALUES (15, 'Стручни студии за програмирање', '3');

Commands completed successfully.

Total execution time: 00:00:00.045

In [85]:
SET NOCOUNT ON;

DELETE FROM Courses
    WHERE name = N'Калкулус'

DELETE FROM Courses
    WHERE name = N'Одржливи и енергетски ефикасни компјутерски системи'

DELETE FROM Courses
    WHERE name = N'Пресметковна биологија'

DELETE FROM Courses
    WHERE name = N'Безжични мултимедиски системи'

Commands completed successfully.

Total execution time: 00:00:00.074

In [86]:
SET NOCOUNT ON;

UPDATE Courses
    SET coursesCode = 'F23L1S026', name = N'F23L2S026 Маркетинг'
    WHERE name = N'Маркетинг';

UPDATE Courses
    SET coursesCode = 'F23L1S052', name = N'Е-учење'
    WHERE name = N'ИТ системи за учење';

UPDATE Courses
    SET coursesCode = 'F23L1S066', name = N'F23L1S066 Основи на сајбер безбедноста'
    WHERE name = N'Основи на сајбер безбедноста';

UPDATE Courses
    SET coursesCode = 'F23L2S051', name = N'Информатичко размислување во образованието'
    WHERE name = N'ИКТ во образованието';

UPDATE Courses
    SET coursesCode = 'F23L2W165', name = N'Управување со техничка поддршка'
    WHERE name = N'Управување со техничката поддршка';

UPDATE Courses
    SET coursesCode = 'F23L3S039', name = N'Основи на теоријата на компјутерските науки'
    WHERE name = N'Формални јазици и автомати';

UPDATE Courses
    SET coursesCode = 'F23L3S106', name = N'Откривање знаење со длабоко учење'
    WHERE name = N'Интелигентни информациски системи';

UPDATE Courses
    SET coursesCode = 'F23L3S125', name = N'Мерење и анализа на сообраќај'
    WHERE name = N'Мерење и анализа на интернет сообраќај';

UPDATE Courses
    SET coursesCode = 'F23L3S150', name = N'Податочно рударење'
    WHERE name = N'Податочно рударство';

UPDATE Courses
    SET coursesCode = 'F23L3S153', name = N'Вештачка интелигенција за игри'
    WHERE name = N'Процесна роботика';

UPDATE Courses
    SET coursesCode = 'F23L3S163', name = N'Автоматизирање на процеси во машинско учење'
    WHERE name = N'Статистичко моделирање';

UPDATE Courses
    SET coursesCode = 'F23L3W027', name = N'Менаџмент информациски системи'
    WHERE name = N'Менаџмент на информациски системи';

UPDATE Courses
    SET coursesCode = 'F23L3W065', name = N'Сајбер безбедност'
    WHERE name = N'Мрежна безбедност';

UPDATE Courses
    SET coursesCode = 'F23L3W074', name = N'Администрација на бази податоци'
    WHERE name = N'Администрација на бази на податоци';

UPDATE Courses
    SET coursesCode = 'F23L3W103', name = N'Имплементација на софтверски системи со слободен и отворен код'
    WHERE name = N'Имплементација на системи со слободен и отворен код';

UPDATE Courses
    SET coursesCode = 'F23L3S115', name = N'Компјутерски звук, музика и говор'
    WHERE name = N'Компјутерски звук, говор и музика';

UPDATE Courses
    SET coursesCode = 'F23L3W154', name = N'Вовед во рударење на масивни податоци'
    WHERE name = N'Рударење на масивни податоци';

UPDATE Courses
    SET coursesCode = 'F23L3W161', name = N'Теорија на графови и социјални мрежи'
    WHERE name = N'Социјални мрежи и медиуми';

Commands completed successfully.

Total execution time: 00:00:00.224

In [87]:
SET NOCOUNT ON;

UPDATE Courses
    SET coursesCode = 'F23L3W037'
    WHERE name = N'Паралелно и дистрибуирано процесирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3W092'
    WHERE name = N'Дигитална постпродукција'; 

UPDATE Courses
    SET coursesCode = 'F23L3S022'
    WHERE name = N'Управување со ИКТ проекти'; 

UPDATE Courses
    SET coursesCode = 'F23L3W074'
    WHERE name = N'Администрација на бази податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3S039'
    WHERE name = N'Основи на теоријата на компјутерските науки'; 

UPDATE Courses
    SET coursesCode = 'F23L3W038'
    WHERE name = N'Програмски парадигми'; 

UPDATE Courses
    SET coursesCode = 'F23L3S168'
    WHERE name = N'Дипломска работа'; 

UPDATE Courses
    SET coursesCode = 'F23L3W068'
    WHERE name = N'Пресметување во облак'; 

UPDATE Courses
    SET coursesCode = 'F23L3S070'
    WHERE name = N'Македонски јазик'; 

UPDATE Courses
    SET coursesCode = 'F23L3S100'
    WHERE name = N'Деловна пракса'; 

UPDATE Courses
    SET coursesCode = 'F23L3S070'
    WHERE name = N'Македонски јазик'; 

UPDATE Courses
    SET coursesCode = 'F23L2S097'
    WHERE name = N'Дизајн на алгоритми'; 

UPDATE Courses
    SET coursesCode = 'F23L1S120'
    WHERE name = N'Креативни вештини за решавање проблеми'; 

UPDATE Courses
    SET coursesCode = 'F23L3W134'
    WHERE name = N'Мултимедиски мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L2S061'
    WHERE name = N'Безжични и мобилни системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W009'
    WHERE name = N'Дизајн и архитектура на софтвер'; 

UPDATE Courses
    SET coursesCode = 'F23L3S131'
    WHERE name = N'Моделирање и симулација'; 

UPDATE Courses
    SET coursesCode = 'F23L3W126'
    WHERE name = N'Методологија на истражувањето во ИКТ'; 

UPDATE Courses
    SET coursesCode = 'F23L3W133'
    WHERE name = N'Мрежна и мобилна форензика'; 

UPDATE Courses
    SET coursesCode = 'F23L3S028'
    WHERE name = N'Претприемништво'; 

UPDATE Courses
    SET coursesCode = 'F23L1S016'
    WHERE name = N'Објектно-ориентирано програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3S101'
    WHERE name = N'Етичко хакирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3W123'
    WHERE name = N'Машинска визија'; 

UPDATE Courses
    SET coursesCode = 'F23L3W072'
    WHERE name = N'Автономна роботика'; 

UPDATE Courses
    SET coursesCode = 'F23L3S159'
    WHERE name = N'Софтверски дефинирана безбедност'; 

UPDATE Courses
    SET coursesCode = 'F23L3W001'
    WHERE name = N'Математика 3'; 

UPDATE Courses
    SET coursesCode = 'F23L2S029'
    WHERE name = N'Софтверско инженерство'; 

UPDATE Courses
    SET coursesCode = 'F23L3S091'
    WHERE name = N'Географски информациски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L1S116'
    WHERE name = N'Компјутерски компоненти'; 

UPDATE Courses
    SET coursesCode = 'F23L2W055'
    WHERE name = N'Мултимедијални технологии'; 

UPDATE Courses
    SET coursesCode = 'F23L3S063'
    WHERE name = N'Дизајн на компјутерски мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W133'
    WHERE name = N'Мрежна и мобилна форензика'; 

UPDATE Courses
    SET coursesCode = 'F23L2W006'
    WHERE name = N'Веројатност и статистика'; 

UPDATE Courses
    SET coursesCode = 'F23L2S017'
    WHERE name = N'Оперативни системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W079'
    WHERE name = N'Веб базирани системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W121'
    WHERE name = N'Блоковски вериги и криптовалути'; 

UPDATE Courses
    SET coursesCode = 'F23L3S086'
    WHERE name = N'Вовед во когнитивни науки'; 

UPDATE Courses
    SET coursesCode = 'F23L3W092'
    WHERE name = N'Дигитална постпродукција'; 

UPDATE Courses
    SET coursesCode = 'F23L3W065'
    WHERE name = N'Сајбер безбедност'; 

UPDATE Courses
    SET coursesCode = 'F23L1S120'
    WHERE name = N'Креативни вештини за решавање проблеми'; 

UPDATE Courses
    SET coursesCode = 'F23L3S010'
    WHERE name = N'Дизајн на интеракцијата човек-компјутер'; 

UPDATE Courses
    SET coursesCode = 'F23L3S138'
    WHERE name = N'Напредни бази на податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3S071'
    WHERE name = N'Психологија на училишна возраст'; 

UPDATE Courses
    SET coursesCode = 'F23L3W089'
    WHERE name = N'Вовед во препознавање на облици'; 

UPDATE Courses
    SET coursesCode = 'F23L3W068'
    WHERE name = N'Пресметување во облак'; 

UPDATE Courses
    SET coursesCode = 'F23L3W024'
    WHERE name = N'Веб програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L2W109'
    WHERE name = N'Интернет програмирање на клиентска страна'; 

UPDATE Courses
    SET coursesCode = 'F23L3S138'
    WHERE name = N'Напредни бази на податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L2S084'
    WHERE name = N'Вовед во екоинформатиката'; 

UPDATE Courses
    SET coursesCode = 'F23L3W038'
    WHERE name = N'Програмски парадигми'; 

UPDATE Courses
    SET coursesCode = 'F23L2W049'
    WHERE name = N'Физика'; 

UPDATE Courses
    SET coursesCode = 'F23L3W079'
    WHERE name = N'Веб базирани системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W121'
    WHERE name = N'Блоковски вериги и криптовалути'; 

UPDATE Courses
    SET coursesCode = 'F23L1S120'
    WHERE name = N'Креативни вештини за решавање проблеми'; 

UPDATE Courses
    SET coursesCode = 'F23L3S102'
    WHERE name = N'ИКТ за развој'; 

UPDATE Courses
    SET coursesCode = 'F23L2W055'
    WHERE name = N'Мултимедијални технологии'; 

UPDATE Courses
    SET coursesCode = 'F23L3S102'
    WHERE name = N'ИКТ за развој'; 

UPDATE Courses
    SET coursesCode = 'F23L3W065'
    WHERE name = N'Сајбер безбедност'; 

UPDATE Courses
    SET coursesCode = 'F23L2W167'
    WHERE name = N'Шаблони за дизајн на кориснички интерфејси'; 

UPDATE Courses
    SET coursesCode = 'F23L2S042'
    WHERE name = N'Електрични кола'; 

UPDATE Courses
    SET coursesCode = 'F23L2S034'
    WHERE name = N'Калкулус 2'; 

UPDATE Courses
    SET coursesCode = 'F23L3S047'
    WHERE name = N'Процесирање на сигналите'; 

UPDATE Courses
    SET coursesCode = 'F23L1W005'
    WHERE name = N'Бизнис и менаџмент'; 

UPDATE Courses
    SET coursesCode = 'F23L3S071'
    WHERE name = N'Психологија на училишна возраст'; 

UPDATE Courses
    SET coursesCode = 'F23L2W046'
    WHERE name = N'Компјутерски мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L1S116'
    WHERE name = N'Компјутерски компоненти'; 

UPDATE Courses
    SET coursesCode = 'F23L2W147'
    WHERE name = N'Основи на комуникациски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W008'
    WHERE name = N'Вовед во науката за податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3S010'
    WHERE name = N'Дизајн на интеракцијата човек-компјутер'; 

UPDATE Courses
    SET coursesCode = 'F23L3S062'
    WHERE name = N'Виртуелизација'; 

UPDATE Courses
    SET coursesCode = 'F23L3S036'
    WHERE name = N'Машинско учење'; 

UPDATE Courses
    SET coursesCode = 'F23L3S107'
    WHERE name = N'Интелигентни системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W001'
    WHERE name = N'Математика 3'; 

UPDATE Courses
    SET coursesCode = 'F23L3W038'
    WHERE name = N'Програмски парадигми'; 

UPDATE Courses
    SET coursesCode = 'F23L3W064'
    WHERE name = N'Дистрибуирани системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W060'
    WHERE name = N'Администрација на системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W140'
    WHERE name = N'Напредно програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3S040'
    WHERE name = N'Вградливи микропроцесорски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S019'
    WHERE name = N'Софтверски квалитет и тестирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3S059'
    WHERE name = N'Администрација на мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L2S110'
    WHERE name = N'Интернет технологии'; 

UPDATE Courses
    SET coursesCode = 'F23L3W117'
    WHERE name = N'Компјутерски поддржано производство'; 

UPDATE Courses
    SET coursesCode = 'F23L2W167'
    WHERE name = N'Шаблони за дизајн на кориснички интерфејси'; 

UPDATE Courses
    SET coursesCode = 'F23L3W008'
    WHERE name = N'Вовед во науката за податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3W126'
    WHERE name = N'Методологија на истражувањето во ИКТ'; 

UPDATE Courses
    SET coursesCode = 'F23L3S130'
    WHERE name = N'Моделирање и менаџирање на бизнис процеси'; 

UPDATE Courses
    SET coursesCode = 'F23L3S022'
    WHERE name = N'Управување со ИКТ проекти'; 

UPDATE Courses
    SET coursesCode = 'F23L3S047'
    WHERE name = N'Процесирање на сигналите'; 

UPDATE Courses
    SET coursesCode = 'F23L1W003'
    WHERE name = N'Вовед во бази на податоци и SQL'; 

UPDATE Courses
    SET coursesCode = 'F23L2S099'
    WHERE name = N'Е-влада'; 

UPDATE Courses
    SET coursesCode = 'F23L3S135'
    WHERE name = N'Мултимедиски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S022'
    WHERE name = N'Управување со ИКТ проекти'; 

UPDATE Courses
    SET coursesCode = 'F23L1S026'
    WHERE name = N'F23L2S026 Маркетинг'; 

UPDATE Courses
    SET coursesCode = 'F23L2S095'
    WHERE name = N'Дигитално процесирање на слика'; 

UPDATE Courses
    SET coursesCode = 'F23L3W137'
    WHERE name = N'Напредна интеракција човек компјутер'; 

UPDATE Courses
    SET coursesCode = 'F23L3W089'
    WHERE name = N'Вовед во препознавање на облици'; 

UPDATE Courses
    SET coursesCode = 'F23L3S111'
    WHERE name = N'Инфраструктурно програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L2S114'
    WHERE name = N'Компјутерска графика'; 

UPDATE Courses
    SET coursesCode = 'F23L3S028'
    WHERE name = N'Претприемништво'; 

UPDATE Courses
    SET coursesCode = 'F23L2S110'
    WHERE name = N'Интернет технологии'; 

UPDATE Courses
    SET coursesCode = 'F23L3W123'
    WHERE name = N'Машинска визија'; 

UPDATE Courses
    SET coursesCode = 'F23L3S073'
    WHERE name = N'Агентно-базирани системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W064'
    WHERE name = N'Дистрибуирани системи'; 

UPDATE Courses
    SET coursesCode = 'F23L1S146'
    WHERE name = N'Основи на Веб дизајн'; 

UPDATE Courses
    SET coursesCode = 'F23L3S144'
    WHERE name = N'Операциони истражувања'; 

UPDATE Courses
    SET coursesCode = 'F23L3S069'
    WHERE name = N'Адаптивни и интерактивни веб информациски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W009'
    WHERE name = N'Дизајн и архитектура на софтвер'; 

UPDATE Courses
    SET coursesCode = 'F23L3W044'
    WHERE name = N'Компјутерска електроника'; 

UPDATE Courses
    SET coursesCode = 'F23L3W058'
    WHERE name = N'Самостоен проект'; 

UPDATE Courses
    SET coursesCode = 'F23L3W075'
    WHERE name = N'Анализа и дизајн на ИС'; 

UPDATE Courses
    SET coursesCode = 'F23L3S087'
    WHERE name = N'Вовед во мрежна наука'; 

UPDATE Courses
    SET coursesCode = 'F23L2W031'
    WHERE name = N'Дискретни структури 1'; 

UPDATE Courses
    SET coursesCode = 'F23L3W105'
    WHERE name = N'Иновации во ИКТ'; 

UPDATE Courses
    SET coursesCode = 'F23L3S093'
    WHERE name = N'Дигитална форензика'; 

UPDATE Courses
    SET coursesCode = 'F23L3W044'
    WHERE name = N'Компјутерска електроника'; 

UPDATE Courses
    SET coursesCode = 'F23L1S146'
    WHERE name = N'Основи на Веб дизајн'; 

UPDATE Courses
    SET coursesCode = 'F23L3W158'
    WHERE name = N'Современи компјутерски архитектури'; 

UPDATE Courses
    SET coursesCode = 'F23L2S042'
    WHERE name = N'Електрични кола'; 

UPDATE Courses
    SET coursesCode = 'F23L1S003'
    WHERE name = N'Архитектура и организација на компјутери'; 

UPDATE Courses
    SET coursesCode = 'F23L2S002'
    WHERE name = N'Анализа на софтверските барања'; 

UPDATE Courses
    SET coursesCode = 'F23L1S052'
    WHERE name = N'Е-учење'; 

UPDATE Courses
    SET coursesCode = 'F23L3W056'
    WHERE name = N'Персонализирано учење'; 

UPDATE Courses
    SET coursesCode = 'F23L2S032'
    WHERE name = N'Дискретни структури 2'; 

UPDATE Courses
    SET coursesCode = 'F23L3S039'
    WHERE name = N'Основи на теоријата на компјутерските науки'; 

UPDATE Courses
    SET coursesCode = 'F23L3S070'
    WHERE name = N'Македонски јазик'; 

UPDATE Courses
    SET coursesCode = 'F23L3S111'
    WHERE name = N'Инфраструктурно програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L1S026'
    WHERE name = N'F23L2S026 Маркетинг'; 

UPDATE Courses
    SET coursesCode = 'F23L3W142'
    WHERE name = N'Обработка на природните јазици'; 

UPDATE Courses
    SET coursesCode = 'F23L3W103'
    WHERE name = N'Имплементација на софтверски системи со слободен и отворен код'; 

UPDATE Courses
    SET coursesCode = 'F23L3S141'
    WHERE name = N'Неструктурирани бази на податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3W027'
    WHERE name = N'Менаџмент информациски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L2S015'
    WHERE name = N'Објектно ориентирана анализа и дизајн'; 

UPDATE Courses
    SET coursesCode = 'F23L2S017'
    WHERE name = N'Оперативни системи'; 

UPDATE Courses
    SET coursesCode = 'F23L2W003'
    WHERE name = N'Избрани теми од математика'; 

UPDATE Courses
    SET coursesCode = 'F23L3W148'
    WHERE name = N'Основи на роботиката'; 

UPDATE Courses
    SET coursesCode = 'F23L2W006'
    WHERE name = N'Веројатност и статистика'; 

UPDATE Courses
    SET coursesCode = 'F23L3S069'
    WHERE name = N'Адаптивни и интерактивни веб информациски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S057'
    WHERE name = N'Работа со надарени ученици'; 

UPDATE Courses
    SET coursesCode = 'F23L3W053'
    WHERE name = N'Компјутерска етика'; 

UPDATE Courses
    SET coursesCode = 'F23L3W136'
    WHERE name = N'Напреден веб дизајн'; 

UPDATE Courses
    SET coursesCode = 'F23L1W018'
    WHERE name = N'Професионални вештини'; 

UPDATE Courses
    SET coursesCode = 'F23L2S082'
    WHERE name = N'Визуелно програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L2S090'
    WHERE name = N'Вовед во случајни процеси'; 

UPDATE Courses
    SET coursesCode = 'F23L3W145'
    WHERE name = N'Оптички мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S106'
    WHERE name = N'Откривање знаење со длабоко учење'; 

UPDATE Courses
    SET coursesCode = 'F23L2W096'
    WHERE name = N'Дигитизација'; 

UPDATE Courses
    SET coursesCode = 'F23L3W161'
    WHERE name = N'Теорија на графови и социјални мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W148'
    WHERE name = N'Основи на роботиката'; 

UPDATE Courses
    SET coursesCode = 'F23L3S080'
    WHERE name = N'Веб пребарувачки системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S112'
    WHERE name = N'Програмски јазици и компајлери'; 

UPDATE Courses
    SET coursesCode = 'F23L3S132'
    WHERE name = N'Модерни трендови во роботика'; 

UPDATE Courses
    SET coursesCode = 'F23L3S111'
    WHERE name = N'Инфраструктурно програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3W162'
    WHERE name = N'Квантно пресметување'; 

UPDATE Courses
    SET coursesCode = 'F23L2W067'
    WHERE name = N'Основи на теоријата на информации'; 

UPDATE Courses
    SET coursesCode = 'F23L3S127'
    WHERE name = N'Мобилни апликации'; 

UPDATE Courses
    SET coursesCode = 'F23L3W004'
    WHERE name = N'Бази на податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3W088'
    WHERE name = N'Вовед во паметни градови'; 

UPDATE Courses
    SET coursesCode = 'F23L3W048'
    WHERE name = N'Софтвер за вградливи системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W004'
    WHERE name = N'Бази на податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3S163'
    WHERE name = N'Автоматизирање на процеси во машинско учење'; 

UPDATE Courses
    SET coursesCode = 'F23L2S051'
    WHERE name = N'Информатичко размислување во образованието'; 

UPDATE Courses
    SET coursesCode = 'F23L2W100'
    WHERE name = N'Економија за ИКТ инженери'; 

UPDATE Courses
    SET coursesCode = 'F23L3S019'
    WHERE name = N'Софтверски квалитет и тестирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3S122'
    WHERE name = N'Криптографија'; 

UPDATE Courses
    SET coursesCode = 'F23L1S023'
    WHERE name = N'Бизнис статистика'; 

UPDATE Courses
    SET coursesCode = 'F23L2S015'
    WHERE name = N'Објектно ориентирана анализа и дизајн'; 

UPDATE Courses
    SET coursesCode = 'F23L3S094'
    WHERE name = N'Дигитални библиотеки'; 

UPDATE Courses
    SET coursesCode = 'F23L3S080'
    WHERE name = N'Веб пребарувачки системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W152'
    WHERE name = N'Програмирање на видео игри'; 

UPDATE Courses
    SET coursesCode = 'F23L3S162'
    WHERE name = N'Споделување и пресметување во толпа'; 

UPDATE Courses
    SET coursesCode = 'F23L3S062'
    WHERE name = N'Виртуелизација'; 

UPDATE Courses
    SET coursesCode = 'F23L3S157'
    WHERE name = N'Складови на податоци и аналитичка обработка'; 

UPDATE Courses
    SET coursesCode = 'F23L3W129'
    WHERE name = N'Мобилни платформи и програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3W021'
    WHERE name = N'Тимски проект'; 

UPDATE Courses
    SET coursesCode = 'F23L2S164'
    WHERE name = N'Теорија на информации со дигитални комуникации'; 

UPDATE Courses
    SET coursesCode = 'F23L3S139'
    WHERE name = N'Web3 апликации'; 

UPDATE Courses
    SET coursesCode = 'F23L3W081'
    WHERE name = N'Визуелизација'; 

UPDATE Courses
    SET coursesCode = 'F23L3W035'
    WHERE name = N'Линеарна алгебра и примени'; 

UPDATE Courses
    SET coursesCode = 'F23L2S124'
    WHERE name = N'Медиуми и комуникации'; 

UPDATE Courses
    SET coursesCode = 'F23L3W156'
    WHERE name = N'Системи за поддршка при одлучувањето'; 

UPDATE Courses
    SET coursesCode = 'F23L3S078'
    WHERE name = N'Биолошки инспирирано пресметување'; 

UPDATE Courses
    SET coursesCode = 'F23L3S100'
    WHERE name = N'Деловна пракса'; 

UPDATE Courses
    SET coursesCode = 'F23L3S131'
    WHERE name = N'Моделирање и симулација'; 

UPDATE Courses
    SET coursesCode = 'F23L2W070'
    WHERE name = N'Педагогија'; 

UPDATE Courses
    SET coursesCode = 'F23L3W145'
    WHERE name = N'Оптички мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S012'
    WHERE name = N'Интегрирани системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W043'
    WHERE name = N'Информациска безбедност'; 

UPDATE Courses
    SET coursesCode = 'F23L2S164'
    WHERE name = N'Теорија на информации со дигитални комуникации'; 

UPDATE Courses
    SET coursesCode = 'F23L3W108'
    WHERE name = N'Интернет на нештата'; 

UPDATE Courses
    SET coursesCode = 'F23L3S101'
    WHERE name = N'Етичко хакирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3W085'
    WHERE name = N'Вовед во биоинформатиката'; 

UPDATE Courses
    SET coursesCode = 'F23L2W165'
    WHERE name = N'Управување со техничка поддршка'; 

UPDATE Courses
    SET coursesCode = 'F23L3S141'
    WHERE name = N'Неструктурирани бази на податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3W105'
    WHERE name = N'Иновации во ИКТ'; 

UPDATE Courses
    SET coursesCode = 'F23L3S012'
    WHERE name = N'Интегрирани системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W074'
    WHERE name = N'Администрација на бази податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L1S116'
    WHERE name = N'Компјутерски компоненти'; 

UPDATE Courses
    SET coursesCode = 'F23L3S166'
    WHERE name = N'Учење на далечина'; 

UPDATE Courses
    SET coursesCode = 'F23L1S052'
    WHERE name = N'Е-учење'; 

UPDATE Courses
    SET coursesCode = 'F23L3S107'
    WHERE name = N'Интелигентни системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W152'
    WHERE name = N'Програмирање на видео игри'; 

UPDATE Courses
    SET coursesCode = 'F23L2S066'
    WHERE name = N'F23L1S066 Основи на сајбер безбедноста'; 

UPDATE Courses
    SET coursesCode = 'F23L3S160'
    WHERE name = N'Софтверски дефинирани мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S078'
    WHERE name = N'Биолошки инспирирано пресметување'; 

UPDATE Courses
    SET coursesCode = 'F23L3W048'
    WHERE name = N'Софтвер за вградливи системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S139'
    WHERE name = N'Web3 апликации'; 

UPDATE Courses
    SET coursesCode = 'F23L3S086'
    WHERE name = N'Вовед во когнитивни науки'; 

UPDATE Courses
    SET coursesCode = 'F23L2W167'
    WHERE name = N'Шаблони за дизајн на кориснички интерфејси'; 

UPDATE Courses
    SET coursesCode = 'F23L3S155'
    WHERE name = N'Сервисно ориентирани архитектури'; 

UPDATE Courses
    SET coursesCode = 'F23L3S112'
    WHERE name = N'Програмски јазици и компајлери'; 

UPDATE Courses
    SET coursesCode = 'F23L3S150'
    WHERE name = N'Податочно рударење'; 

UPDATE Courses
    SET coursesCode = 'F23L3W076'
    WHERE name = N'Вовед во анализа на временските серии'; 

UPDATE Courses
    SET coursesCode = 'F23L2W109'
    WHERE name = N'Интернет програмирање на клиентска страна'; 

UPDATE Courses
    SET coursesCode = 'F23L2S066'
    WHERE name = N'F23L1S066 Основи на сајбер безбедноста'; 

UPDATE Courses
    SET coursesCode = 'F23L3W064'
    WHERE name = N'Дистрибуирани системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W027'
    WHERE name = N'Менаџмент информациски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L2W046'
    WHERE name = N'Компјутерски мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S127'
    WHERE name = N'Мобилни апликации'; 

UPDATE Courses
    SET coursesCode = 'F23L2S119'
    WHERE name = N'Концепти на информатичко општество'; 

UPDATE Courses
    SET coursesCode = 'F23L3S149'
    WHERE name = N'Паралелно програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3S144'
    WHERE name = N'Операциони истражувања'; 

UPDATE Courses
    SET coursesCode = 'F23L3S160'
    WHERE name = N'Софтверски дефинирани мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W056'
    WHERE name = N'Персонализирано учење'; 

UPDATE Courses
    SET coursesCode = 'F23L2S011'
    WHERE name = N'Дискретна математика'; 

UPDATE Courses
    SET coursesCode = 'F23L3S025'
    WHERE name = N'Електронска и мобилна трговија'; 

UPDATE Courses
    SET coursesCode = 'F23L3S063'
    WHERE name = N'Дизајн на компјутерски мрежи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S057'
    WHERE name = N'Работа со надарени ученици'; 

UPDATE Courses
    SET coursesCode = 'F23L3W060'
    WHERE name = N'Администрација на системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3S159'
    WHERE name = N'Софтверски дефинирана безбедност'; 

UPDATE Courses
    SET coursesCode = 'F23L3W129'
    WHERE name = N'Мобилни платформи и програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3S115'
    WHERE name = N'Компјутерски звук, музика и говор'; 

UPDATE Courses
    SET coursesCode = 'F23L1S052'
    WHERE name = N'Е-учење'; 

UPDATE Courses
    SET coursesCode = 'F23L3W053'
    WHERE name = N'Компјутерска етика'; 

UPDATE Courses
    SET coursesCode = 'F23L3W128'
    WHERE name = N'Мобилни информациски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L2S051'
    WHERE name = N'Информатичко размислување во образованието'; 

UPDATE Courses
    SET coursesCode = 'F23L3S106'
    WHERE name = N'Откривање знаење со длабоко учење'; 

UPDATE Courses
    SET coursesCode = 'F23L1S146'
    WHERE name = N'Основи на Веб дизајн'; 

UPDATE Courses
    SET coursesCode = 'F23L1W007'
    WHERE name = N'Вовед во компјутерските науки'; 

UPDATE Courses
    SET coursesCode = 'F23L2S015'
    WHERE name = N'Објектно ориентирана анализа и дизајн'; 

UPDATE Courses
    SET coursesCode = 'F23L3S153'
    WHERE name = N'Вештачка интелигенција за игри'; 

UPDATE Courses
    SET coursesCode = 'F23L3W108'
    WHERE name = N'Интернет на нештата'; 

UPDATE Courses
    SET coursesCode = 'F23L3S025'
    WHERE name = N'Електронска и мобилна трговија'; 

UPDATE Courses
    SET coursesCode = 'F23L3W098'
    WHERE name = N'Дистрибуирано складирање на податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3W037'
    WHERE name = N'Паралелно и дистрибуирано процесирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3W154'
    WHERE name = N'Вовед во рударење на масивни податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3W088'
    WHERE name = N'Вовед во паметни градови'; 

UPDATE Courses
    SET coursesCode = 'F23L2S029'
    WHERE name = N'Софтверско инженерство'; 

UPDATE Courses
    SET coursesCode = 'F23L2S051'
    WHERE name = N'Информатичко размислување во образованието'; 

UPDATE Courses
    SET coursesCode = 'F23L3W050'
    WHERE name = N'Дизајн на образовен софтвер'; 

UPDATE Courses
    SET coursesCode = 'F23L2W001'
    WHERE name = N'Алгоритми и податочни структури'; 

UPDATE Courses
    SET coursesCode = 'F23L3S113'
    WHERE name = N'Компјутерска анимација'; 

UPDATE Courses
    SET coursesCode = 'F23L1S026'
    WHERE name = N'F23L2S026 Маркетинг'; 

UPDATE Courses
    SET coursesCode = 'F23L1S052'
    WHERE name = N'Е-учење'; 

UPDATE Courses
    SET coursesCode = 'F23L3W076'
    WHERE name = N'Вовед во анализа на временските серии'; 

UPDATE Courses
    SET coursesCode = 'F23L3S083'
    WHERE name = N'Виртуелна реалност'; 

UPDATE Courses
    SET coursesCode = 'F23L3S054'
    WHERE name = N'Методика на информатиката'; 

UPDATE Courses
    SET coursesCode = 'F23L3S118'
    WHERE name = N'Континуирана интеграција и испорака'; 

UPDATE Courses
    SET coursesCode = 'F23L1S045'
    WHERE name = N'Компјутерски архитектури'; 

UPDATE Courses
    SET coursesCode = 'F23L3W154'
    WHERE name = N'Вовед во рударење на масивни податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L2S030'
    WHERE name = N'Вештачка интелигенција'; 

UPDATE Courses
    SET coursesCode = 'F23L3W140'
    WHERE name = N'Напредно програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3W136'
    WHERE name = N'Напреден веб дизајн'; 

UPDATE Courses
    SET coursesCode = 'F23L1W020'
    WHERE name = N'Структурно програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L2S030'
    WHERE name = N'Вештачка интелигенција'; 

UPDATE Courses
    SET coursesCode = 'F23L2W014'
    WHERE name = N'Компјутерски мрежи и безбедност'; 

UPDATE Courses
    SET coursesCode = 'F23L3S130'
    WHERE name = N'Моделирање и менаџирање на бизнис процеси'; 

UPDATE Courses
    SET coursesCode = 'F23L3S157'
    WHERE name = N'Складови на податоци и аналитичка обработка'; 

UPDATE Courses
    SET coursesCode = 'F23L2W104'
    WHERE name = N'Инженерска математика'; 

UPDATE Courses
    SET coursesCode = 'F23L3W156'
    WHERE name = N'Системи за поддршка при одлучувањето'; 

UPDATE Courses
    SET coursesCode = 'F23L3S054'
    WHERE name = N'Методика на информатиката'; 

UPDATE Courses
    SET coursesCode = 'F23L3W033'
    WHERE name = N'Тестирање на софтвер'; 

UPDATE Courses
    SET coursesCode = 'F23L2W033'
    WHERE name = N'Калкулус 1'; 

UPDATE Courses
    SET coursesCode = 'F23L2S002'
    WHERE name = N'Анализа на софтверските барања'; 

UPDATE Courses
    SET coursesCode = 'F23L3W098'
    WHERE name = N'Дистрибуирано складирање на податоци'; 

UPDATE Courses
    SET coursesCode = 'F23L3S125'
    WHERE name = N'Мерење и анализа на сообраќај'; 

UPDATE Courses
    SET coursesCode = 'F23L3S036'
    WHERE name = N'Машинско учење'; 

UPDATE Courses
    SET coursesCode = 'F23L3W050'
    WHERE name = N'Дизајн на образовен софтвер'; 

UPDATE Courses
    SET coursesCode = 'F23L2W100'
    WHERE name = N'Економија за ИКТ инженери'; 

UPDATE Courses
    SET coursesCode = 'F23L3W085'
    WHERE name = N'Вовед во биоинформатиката'; 

UPDATE Courses
    SET coursesCode = 'F23L3S040'
    WHERE name = N'Вградливи микропроцесорски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W043'
    WHERE name = N'Информациска безбедност'; 

UPDATE Courses
    SET coursesCode = 'F23L2S124'
    WHERE name = N'Медиуми и комуникации'; 

UPDATE Courses
    SET coursesCode = 'F23L3W075'
    WHERE name = N'Анализа и дизајн на ИС'; 

UPDATE Courses
    SET coursesCode = 'F23L3W108'
    WHERE name = N'Интернет на нештата'; 

UPDATE Courses
    SET coursesCode = 'F23L3W128'
    WHERE name = N'Мобилни информациски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W162'
    WHERE name = N'Квантно пресметување'; 

UPDATE Courses
    SET coursesCode = 'F23L3W200'
    WHERE name = N'Сензорски системи'; 

UPDATE Courses
    SET coursesCode = 'F23L3W068'
    WHERE name = N'Пресметување во облак'; 

UPDATE Courses
    SET coursesCode = 'F23L2S042'
    WHERE name = N'Електрични кола'; 

UPDATE Courses
    SET coursesCode = 'F23L3W024'
    WHERE name = N'Веб програмирање'; 

UPDATE Courses
    SET coursesCode = 'F23L3W103'
    WHERE name = N'Имплементација на софтверски системи со слободен и отворен код'; 

UPDATE Courses
    SET coursesCode = 'F23L2W067'
    WHERE name = N'Основи на теоријата на информации'; 

UPDATE Courses
    SET coursesCode = 'F23L3W072'
    WHERE name = N'Автономна роботика'; 

UPDATE Courses
    SET coursesCode = 'F23L3W126'
    WHERE name = N'Методологија на истражувањето во ИКТ'; 

UPDATE Courses
    SET coursesCode = 'F23L3S166'
    WHERE name = N'Учење на далечина'; 

UPDATE Courses
    SET coursesCode = 'F23L3W033'
    WHERE name = N'Тестирање на софтвер'; 

UPDATE Courses
    SET coursesCode = 'F23L1W004'
    WHERE name = N'Спорт и здравје'; 

UPDATE Courses
    SET coursesCode = 'F23L3S063'
    WHERE name = N'Дизајн на компјутерски мрежи';

UPDATE Courses
    SET coursesCode = 'F23L2W041'
    WHERE name = N'Дизајн на дигитални кола'; 

UPDATE Courses
    SET coursesCode = 'F23L3S162'
    WHERE name = N'Споделување и пресметување во толпа'; 

UPDATE Courses
    SET coursesCode = 'F23L3W048'
    WHERE name = N'Софтвер за вградливи системи'; 

UPDATE Courses
    SET coursesCode = 'F23L2W014'
    WHERE name = N'Компјутерски мрежи и безбедност'; 

UPDATE Courses
    SET coursesCode = 'F23L3S054'
    WHERE name = N'Методика на информатиката'; 

UPDATE Courses
    SET coursesCode = 'F23L3W035'
    WHERE name = N'Линеарна алгебра и примени'; 

UPDATE Courses
    SET coursesCode = 'F23L3S132'
    WHERE name = N'Модерни трендови во роботика'; 

UPDATE Courses
    SET coursesCode = 'F23L3W117'
    WHERE name = N'Компјутерски поддржано производство'; 

UPDATE Courses
    SET coursesCode = 'F23L3W137'
    WHERE name = N'Напредна интеракција човек компјутер'; 

UPDATE Courses
    SET coursesCode = 'F23L3S083'
    WHERE name = N'Виртуелна реалност'; 

UPDATE Courses
    SET coursesCode = 'F23L3S132'
    WHERE name = N'Модерни трендови во роботика'; 

Commands completed successfully.

Total execution time: 00:00:03.892

# **Repopulating Studies Table**

In [88]:
SET NOCOUNT ON;

INSERT INTO Courses (coursesCode, name) VALUES ('F23L2W002', N'Математика 1')
INSERT INTO Courses (coursesCode, name) VALUES ('F23L2S001', N'Математика 2')
INSERT INTO Courses (coursesCode, name) VALUES ('F23L3W001', N'Математика 3')
INSERT INTO Courses (coursesCode, name) VALUES ('F23L2W201', N'Примена на алгоритми и податочни структури')
INSERT INTO Courses (coursesCode, name) VALUES ('F23L3S111', N'Инфраструктурно програмирање')
INSERT INTO Courses (coursesCode, name) VALUES ('F23L2W003', N'Избрани теми од математика')
INSERT INTO Courses (coursesCode, name) VALUES ('F23L3S100', N'Деловна пракса')
INSERT INTO Courses (coursesCode, name) VALUES ('F23L3W033', N'Тестирање на софтвер')
INSERT INTO Courses (coursesCode, name) VALUES ('F23L1W003', N'Вовед во бази на податоци и SQLр')

Commands completed successfully.

Total execution time: 00:00:00.008

In [89]:
SET NOCOUNT ON;

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 0 AND coursesCode = 'F23L2S011'

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 0 AND coursesCode = 'F23L2W001'

UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 0 AND coursesCode = 'F23L1W004' 

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (0, 'F23L2W002', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (0, 'F23L2S001', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (0, 'F23L2W165', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (0, 'F23L2W201', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (0, 'F23L2S110', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (0, 'F23L3S059', 6)

Commands completed successfully.

Total execution time: 00:00:00.013

In [90]:
SET NOCOUNT ON;

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 1 AND coursesCode = 'F23L2W001'
    
UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 1 AND coursesCode = 'F23L1W004' 

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (1, 'F23L2W002', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (1, 'F23L2S001', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (1, 'F23L2W165', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (1, 'F23L2W201', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (1, 'F23L2S110', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (1, 'F23L3S159', 6)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (1, 'F23L3W021', 7)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (1, 'F23L3S111', 8)

Commands completed successfully.

Total execution time: 00:00:00.050

In [91]:
SET NOCOUNT ON;

UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 3 AND coursesCode = 'F23L1W004' 

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (3, 'F23L3W024', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (3, 'F23L3W035', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (3, 'F23L3W037', 5)

Commands completed successfully.

Total execution time: 00:00:00.044

In [92]:
SET NOCOUNT ON;

UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 4 AND coursesCode = 'F23L1W004' 

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (4, 'F23L3W024', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (4, 'F23L3W140', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (4, 'F23L3S157', 6)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (4, 'F23L3W021', 7)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (4, 'F23L3W126', 7)

Commands completed successfully.

Total execution time: 00:00:00.046

In [93]:
SET NOCOUNT ON;

UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 5 AND coursesCode = 'F23L1W004' 

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (5, 'F23L3W043', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (5, 'F23L3W044', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (5, 'F23L3W148', 5)

Commands completed successfully.

Total execution time: 00:00:00.046

In [94]:
SET NOCOUNT ON;

UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 6 AND coursesCode = 'F23L1W004' 

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (6, 'F23L3W148', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (6, 'F23L3S157', 6)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (6, 'F23L3W021', 7)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (6, 'F23L3W108', 7)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (6, 'F23L3S132', 8)

Commands completed successfully.

Total execution time: 00:00:00.048

In [95]:
SET NOCOUNT ON;

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 7 AND coursesCode = 'F23L2S011'

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 7 AND coursesCode = 'F23L2W001'
    
UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 7 AND coursesCode = 'F23L1W004'

UPDATE MandatoryCourseStudies
    SET semester = 4
    WHERE studiesCode = 7 AND coursesCode = 'F23L1S026' 

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (7, 'F23L2W003', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (7, 'F23L1S146', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (7, 'F23L2W201', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (7, 'F23L2W100', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (7, 'F23L3S100', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (7, 'F23L3W008', 5)

Commands completed successfully.

Total execution time: 00:00:00.056

In [96]:
SET NOCOUNT ON;

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 8 AND coursesCode = 'F23L2S011'

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 8 AND coursesCode = 'F23L2W001'
    
UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 8 AND coursesCode = 'F23L1W004'

UPDATE MandatoryCourseStudies
    SET semester = 4
    WHERE studiesCode = 8 AND coursesCode = 'F23L1S026' 

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (8, 'F23L2W003', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (8, 'F23L1S146', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (8, 'F23L2W201', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (8, 'F23L2W100', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (8, 'F23L3S100', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (8, 'F23L3W136', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (8, 'F23L3W033', 7)

Commands completed successfully.

Total execution time: 00:00:00.051

In [97]:
SET NOCOUNT ON;

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 9 AND coursesCode = 'F23L2S011'

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 9 AND coursesCode = 'F23L2W006'
    
UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 9 AND coursesCode = 'F23L1W004'

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (9, 'F23L2W002', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (9, 'F23L2S001', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (9, 'F23L3W001', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (9, 'F23L2S030', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (9, 'F23L3S100', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (9, 'F23L3W008', 5)

Commands completed successfully.

Total execution time: 00:00:00.051

In [98]:
SET NOCOUNT ON;

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 10 AND coursesCode = 'F23L2S011'

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 10 AND coursesCode = 'F23L2W006'
    
UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 10 AND coursesCode = 'F23L1W004'

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (10, 'F23L2W002', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (10, 'F23L2S001', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (10, 'F23L3W001', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (10, 'F23L2S030', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (10, 'F23L3S100', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (10, 'F23L3W140', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (10, 'F23L3S028', 8)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (10, 'F23L3S138', 6)

Commands completed successfully.

Total execution time: 00:00:00.054

In [99]:
SET NOCOUNT ON;

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 11 AND coursesCode = 'F23L2S011'

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 11 AND coursesCode = 'F23L2W006'
    
UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 11 AND coursesCode = 'F23L1W004'

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (11, 'F23L2W002', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (11, 'F23L2S001', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (11, 'F23L3W001', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (11, 'F23L2S030', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (11, 'F23L3S100', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (11, 'F23L3W008', 5)

Commands completed successfully.

Total execution time: 00:00:00.049

In [100]:
SET NOCOUNT ON;

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 12 AND coursesCode = 'F23L2S011'

DELETE FROM MandatoryCourseStudies
    WHERE studiesCode = 12 AND coursesCode = 'F23L2W006'
    
UPDATE MandatoryCourseStudies
    SET semester = 1
    WHERE studiesCode = 12 AND coursesCode = 'F23L1W004'

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (12, 'F23L2W002', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (12, 'F23L2S001', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (12, 'F23L3W001', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (12, 'F23L2S030', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (12, 'F23L3S100', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (12, 'F23L3W140', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (12, 'F23L3S028', 8)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (12, 'F23L3S138', 6)

Commands completed successfully.

Total execution time: 00:00:00.053

In [101]:
SET NOCOUNT ON;

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L1W004', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L1W007', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L1W018', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L1W020', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L2W002', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L2W070', 1)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L1S016', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L1S045', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L1S052', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L1S146', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L2S001', 2)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L2W014', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L2W055', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L2W201', 3)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L1S120', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L2S017', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L2S051', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L2S124', 4)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3W004', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3W050', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3W053', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3W056', 5)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3S057', 6)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3S071', 6)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3S166', 6)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3W058', 7)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3S054', 8)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3S070', 8)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (13, 'F23L3S168', 8)

Commands completed successfully.

Total execution time: 00:00:00.076

In [102]:
SET NOCOUNT ON;

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L1W003', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L1W007', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L1W020', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L2W109', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L2W201', 1)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L1S016', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L1S026', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L1S146', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L2S066', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L3S025', 2)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L2W167', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L3W024', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L3W033', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L3W103', 3)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L2S110', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (14, 'F23L3S100', 4)

Commands completed successfully.

Total execution time: 00:00:00.058

In [103]:
SET NOCOUNT ON;

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L1W003', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L1W007', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L1W020', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L2W109', 1)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L2W201', 1)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L1S016', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L1S026', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L1S146', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L2S066', 2)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L3S025', 2)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L2W167', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L3W024', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L3W033', 3)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L3W103', 3)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L2S110', 4)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L3S100', 4)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L3W021', 5)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L3W027', 5)

INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L2S029', 6)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L3S028', 6)
INSERT INTO MandatoryCourseStudies (studiesCode, coursesCode, semester) VALUES (15, 'F23L3S168', 6)

Commands completed successfully.

Total execution time: 00:00:00.064